Things to try to increase performance: 
-even more data
-models for different genres
-topic model
-ridge regression, lasso regression
-bi grams, n grams
-sentiment analysis

In [321]:
import pandas as pd
from textblob import TextBlob
from textblob import Word
from gensim import corpora, models, similarities
import os
import tempfile
import gensim
from sklearn.linear_model import LinearRegression
import math
import numpy as np
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV

TEMP_FOLDER = tempfile.gettempdir()

In [106]:
links = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/links2.csv', header = None)
text = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/reviewtext.csv', header = None)

In [107]:
links2 = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/links3.csv', header = None)
text2 = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/reviewtext2.csv', header = None)

In [108]:
#prep 2nd set of data, which has lots of links that didn't work

links2[5] = text2
links2.drop_duplicates([5], inplace = True)
links2 = links2[links2[5] != '[]']
links2 = links2[links2[5] != '[]']
links2.dropna(subset = [5],inplace = True)

In [109]:
links[5] = text

In [110]:
links.drop_duplicates([5], inplace = True)


In [111]:
links = pd.concat([links,links2], axis = 0)
links = links.reset_index()
links = links.drop('index', axis = 1)

In [112]:
links.shape

(2903, 6)

In [273]:
#how many of each genre
links.head()
links[3].groupby([3]).count()

,0,1,2,4,5
3,,,,,
Action,511,511,511,511,511
"Action, Adventure",122,122,122,122,122
"Action, Compilation",18,18,18,18,18
"Action, Platformer",2,2,2,2,2
"Action, Puzzle",1,1,1,1,1
"Action, RPG",81,81,81,81,81
"Action, Simulation",4,4,4,4,4
"Action, Strategy",1,1,1,1,1
Adventure,324,324,324,324,324


In [113]:
text = links[5]
text.shape

(2903,)

In [114]:
#lowercase text
text = text.apply(lambda x: " ".join(x.lower() for x in x.split()))
text.head()

0    ["with all of the development muscle behind ea...
1    ['when i first reviewed civilization vi back i...
2    [' ', 'there aren’t many games that make me fe...
3    ['overkill’s ', ' is an earnest attempt to del...
4    ["it's often with rose-colored glasses that i ...
Name: 5, dtype: object

In [115]:
#remove brackets
text = text.apply(lambda x: x[1:-1])
text.head()

0    "with all of the development muscle behind ea ...
1    'when i first reviewed civilization vi back in...
2    ' ', 'there aren’t many games that make me fee...
3    'overkill’s ', ' is an earnest attempt to deli...
4    "it's often with rose-colored glasses that i l...
Name: 5, dtype: object

In [116]:
#remove new line
text = text.replace(r'\\n','', regex=True)
text.head()

0    "with all of the development muscle behind ea ...
1    'when i first reviewed civilization vi back in...
2    ' ', 'there aren’t many games that make me fee...
3    'overkill’s ', ' is an earnest attempt to deli...
4    "it's often with rose-colored glasses that i l...
Name: 5, dtype: object

In [117]:
#remove punc
text = text.str.replace('[^\w\s]','')
text.head()

0    with all of the development muscle behind ea d...
1    when i first reviewed civilization vi back in ...
2      there arent many games that make me feel as ...
3    overkills   is an earnest attempt to deliver a...
4    its often with rosecolored glasses that i look...
Name: 5, dtype: object

In [118]:
#remove stop words
from nltk.corpus import stopwords
stop = stopwords.words('english')
text = text.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
text.head()

0    development muscle behind ea dices massive sho...
1    first reviewed civilization vi back 2016 award...
2    arent many games make feel cool warframe im bu...
3    overkills earnest attempt deliver cooperative ...
4    often rosecolored glasses look back pokemons k...
Name: 5, dtype: object

In [119]:
freq = pd.Series(' '.join(text[0]).split()).value_counts()[-5:]
#remove uncommon words
freq = list(freq.index)
text = text.apply(lambda x: " ".join(x for x in x.split() if x not in freq))
text.head()

0    development muscle behind ea dices massive sho...
1    first reviewed civilization vi back 2016 award...
2    arent many games make feel cool warframe im bu...
3    overkills earnest attempt deliver cooperative ...
4    often rosecolored glasses look back pokemons k...
Name: 5, dtype: object

In [120]:
###remove common words###
freq = pd.Series(' '.join(text[0]).split()).value_counts()[:10]
freq = list(freq.index)
text = text.apply(lambda x: " ".join(x for x in x.split() if x not in freq))
#Visualize
text.head()

0    development muscle behind ea dices massive sho...
1    first reviewed civilization vi back 2016 award...
2    arent many games make feel cool warframe im bu...
3    overkills earnest attempt deliver cooperative ...
4    often rosecolored glasses look back pokemons k...
Name: 5, dtype: object

In [121]:
###Spelling correction###

text[:5].apply(lambda x: str(TextBlob(x).correct()))

###Tokenization (dividing the text into a sequence of words or sentences)###
TextBlob(text[0][1]).words

#Stemming (removal of suffices, like “ing”, “ly”, “s”, etc. by a simple rule-based approach)
#from nltk.stem import PorterStemmer
#st = PorterStemmer()
#train['text'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

#Lemmatization (more effective option than stemming because 
# converts the word into its root word, rather than just stripping the suffices)
#*Preferred to stemming*#


text = text.apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
text.head()

0    development muscle behind ea dice massive shoo...
1    first reviewed civilization vi back 2016 award...
2    arent many game make feel cool warframe im bul...
3    overkill earnest attempt deliver cooperative a...
4    often rosecolored glass look back pokemons kan...
Name: 5, dtype: object

In [122]:
text = text.to_frame()
text['score'] = links[1]

In [123]:
text.head()

,5,score
0,development muscle behind ea dice massive shoo...,7.5
1,first reviewed civilization vi back 2016 award...,9.4
2,arent many game make feel cool warframe im bul...,8.6
3,overkill earnest attempt deliver cooperative a...,5.3
4,often rosecolored glass look back pokemons kan...,8.3


In [295]:
#train val split

trainval = text.sample(800)
train = text.loc[~text.index.isin(trainval.index), :]
y = train['score']
valy = trainval['score']
trainval.drop(['score'], axis = 1, inplace = True)
train.drop(['score'], axis = 1, inplace = True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [296]:
#for train values
textlist = train.values.tolist()

In [297]:
textlist = [str(word) for word in textlist]

In [298]:
texts = [[word for word in document.lower().split() ]
         for document in textlist]

In [299]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=5, no_above=0.6, keep_n=35000)

In [300]:
dictionary.save(os.path.join(TEMP_FOLDER, 'igntfidf.dict'))

In [301]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'igntfidf.mm'), corpus)  # store to disk, for later use

In [302]:
tfidf = models.TfidfModel(corpus)

In [303]:
corpus_tfidf = tfidf[corpus]

In [304]:
all_topics_csr = gensim.matutils.corpus2csc(corpus_tfidf)

In [305]:
all_topics_csr.shape

(13273, 2103)

In [306]:
all_topics_numpy = all_topics_csr.T.toarray()

In [307]:
all_topics_numpy

array([[0.04044892, 0.01415159, 0.04403912, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [308]:
#validation array
vallist = trainval.values.tolist()
vallist = [str(word) for word in vallist]
valtexts = [[word for word in document.lower().split() ]
         for document in vallist]
valcorpus = [dictionary.doc2bow(valtext) for valtext in valtexts]
testcorpus_tfidf = tfidf[valcorpus]
all_topics_csr2 = gensim.matutils.corpus2csc(testcorpus_tfidf)
all_topics_numpytest = all_topics_csr2.T.toarray()

In [309]:
all_topics_numpytest

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.07166795, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [310]:
print(all_topics_numpytest.shape)
print(all_topics_numpy.shape)

(800, 13273)
(2103, 13273)


## XGBoost

In [340]:
yarray = y.values
xgbreg = xgb.XGBRegressor()
xgbreg.fit(all_topics_numpy,yarray)

predictions2 = xgbreg.predict(all_topics_numpytest)
predictionsdf2 = pd.DataFrame(predictions2)
valy = valy.reset_index()
valy.drop(['index'], axis = 1, inplace = True)
predictionsdf2['true'] = valy
predictionsdf2['diff'] = predictionsdf2['true'] - predictionsdf2[0]
predictionsdf2['diff'] = predictionsdf2['true'] - predictionsdf2[0]
predictionsdf2['diff'] = predictionsdf2['diff']**2
diff = predictionsdf2['diff'].values
diff = np.mean(diff)
diff = math.sqrt(diff)
diff

1.0728785941278771

## Lasso Regression

In [338]:
#find best alpha for lasso regression

alphas = [1e-15, 1e-10, 1e-8, 1e-5,1e-4, 1e-3,1e-2, 1, 5, 10]
alphas2 = [1e-5,1.5e-5,1e-4, 1.5e-4,1e-3]
def runlasso(alpha):
    global valy
    yarray = y.values
    lasso = Lasso(alpha = alpha, max_iter=1e5)
    lasso.fit(all_topics_numpy,yarray)

    predictions3 = lasso.predict(all_topics_numpytest)
    predictionsdf3 = pd.DataFrame(predictions3)
    valy = valy.reset_index()
    valy.drop(['index'], axis = 1, inplace = True)
    predictionsdf3['true'] = valy
    predictionsdf3['diff'] = predictionsdf3['true'] - predictionsdf3[0]
    predictionsdf3['diff'] = predictionsdf3['true'] - predictionsdf3[0]
    predictionsdf3['diff'] = predictionsdf3['diff']**2
    diff = predictionsdf3['diff'].values
    diff = np.mean(diff)
    diff = math.sqrt(diff)
    return diff
    
test = [runlasso(x) for x in alphas2]
print(test)
    
    
    

[1.1252147868577373, 1.1126873118616674, 1.0177467770085127, 0.9907478349117224, 1.1576718275231785]


In [339]:
yarray = y.values
lasso = Lasso(alpha = 1.5e-4, max_iter=1e5)
lasso.fit(all_topics_numpy,yarray)

predictions3 = lasso.predict(all_topics_numpytest)
predictionsdf3 = pd.DataFrame(predictions3)
valy = valy.reset_index()
valy.drop(['index'], axis = 1, inplace = True)
predictionsdf3['true'] = valy
predictionsdf3['diff'] = predictionsdf3['true'] - predictionsdf3[0]
predictionsdf3['diff'] = predictionsdf3['true'] - predictionsdf3[0]
predictionsdf3['diff'] = predictionsdf3['diff']**2
diff = predictionsdf3['diff'].values
diff = np.mean(diff)
diff = math.sqrt(diff)
diff


0.9907478349117224

## OLS

In [327]:
yarray = y.values
lm = LinearRegression()
lm.fit(all_topics_numpy,yarray)
predictions = lm.predict(all_topics_numpytest)
predictionsdf = pd.DataFrame(predictions)

In [328]:
predictionsdf.describe()

,0
count,800.000000
mean,7.257459
std,1.230349
min,1.965389
25%,6.399265
50%,7.289813
75%,8.135253
max,10.575431


In [329]:
valy = valy.reset_index()
valy

,index,score
0,0,7.7
1,1,8.0
2,2,9.0
3,3,7.8
4,4,9.0
5,5,6.4
6,6,7.9
7,7,9.2
8,8,7.0
9,9,7.0


In [330]:
valy.drop(['index'], axis = 1, inplace = True)
valy

,score
0,7.7
1,8.0
2,9.0
3,7.8
4,9.0
5,6.4
6,7.9
7,9.2
8,7.0
9,7.0


In [331]:
predictionsdf['true'] = valy
predictionsdf

,0,true
0,7.608641,7.7
1,8.287071,8.0
2,8.508838,9.0
3,8.017991,7.8
4,7.571709,9.0
5,6.604391,6.4
6,8.054136,7.9
7,8.822618,9.2
8,7.870244,7.0
9,6.725158,7.0


In [332]:
predictionsdf['diff'] = predictionsdf['true'] - predictionsdf[0]
predictionsdf

,0,true,diff
0,7.608641,7.7,0.091359
1,8.287071,8.0,-0.287071
2,8.508838,9.0,0.491162
3,8.017991,7.8,-0.217991
4,7.571709,9.0,1.428291
5,6.604391,6.4,-0.204391
6,8.054136,7.9,-0.154136
7,8.822618,9.2,0.377382
8,7.870244,7.0,-0.870244
9,6.725158,7.0,0.274842


In [333]:
#let's also plot histogram of scores




In [334]:
#residual plot here




In [335]:
#RMSE
predictionsdf['diff'] = predictionsdf['true'] - predictionsdf[0]
predictionsdf['diff'] = predictionsdf['diff']**2
diff = predictionsdf['diff'].values
diff = np.mean(diff)
diff = math.sqrt(diff)
diff

#diff = predictionsdf['diff'].values.tolist()
#diff = math.sqrt(np.mean(diff**2))
#diff

1.0528095780414812

In [292]:
arravg = np.mean(yarray)
predictionsdf['avg'] = arravg
predictionsdf['avgdiff'] = predictionsdf['avg'] - predictionsdf['true']
predictionsdf['avgdiff'] = predictionsdf['avgdiff']**2
avgdiff = predictionsdf['avgdiff'].values
avgdiff = np.mean(avgdiff)
avgdiff = math.sqrt(avgdiff)
avgdiff

nan